In [1]:
import pandas as pd

In [3]:
data_fof = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.csv', skiprows=28)
data_subhalo = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.csv', skiprows=37)  
data_subhalo['GalaxyID'] = data_subhalo['GalaxyID'].astype(str)
data_fof['GroupID'] = data_subhalo['GalaxyID'].astype(str)

data_subhalo.to_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.feather')
data_fof.to_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.feather')

/tmp/ipykernel_436946/3252503702.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_fof = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.csv', skiprows=28)
/tmp/ipykernel_436946/3252503702.py:2: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data_subhalo = pd.read_csv('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.csv', skiprows=37)


In [ ]:
data_fof = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_FoF.feather')  
data_fof

In [ ]:
data_subhalo = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/L0100N1504_Subhalo.feather')  
data_subhalo

In [ ]:
same_features = set(data_fof.columns).intersection(set(data_subhalo.columns))
same_features

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [ ]:
fof_unique = data_fof.drop(columns=same_features)
fof_unique.head()

In [ ]:
subhalo_unique = data_subhalo.drop(columns=same_features)
subhalo_unique.head()

In [3]:
data_1 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0025N0376.feather')
data_2 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0025N0752.feather')  
data_3 = pd.read_feather('/mgpfs/home/sarmantara/dmmapper/data/merge_L0100N1504.feather')

print("Data 1:", data_1.columns)
print(data_1.info())
print("\nData 2:", data_2.columns)
print(data_2.info())
print("\nData 3:", data_3.columns)
print(data_3.info())

Data 1: Index(['GroupID', 'SnapNum', 'Redshift', 'RandomNumber', 'GroupMass',
       'GroupCentreOfPotential_x', 'GroupCentreOfPotential_y',
       'GroupCentreOfPotential_z', 'NumOfSubhalos', 'Group_M_Crit200',
       'Group_R_Crit200', 'Group_M_Mean200', 'Group_R_Mean200',
       'Group_M_TopHat200', 'Group_R_TopHat200', 'Group_M_Crit500',
       'Group_R_Crit500', 'Group_M_Mean500', 'Group_R_Mean500',
       'Group_M_Crit2500', 'Group_R_Crit2500', 'Group_R_Mean2500',
       'Group_M_Mean2500', 'GalaxyID', 'DescendantID', 'LastProgID',
       'TopLeafID', 'GroupNumber', 'SubGroupNumber', 'CentreOfMass_x',
       'CentreOfMass_y', 'CentreOfMass_z', 'CentreOfPotential_x',
       'CentreOfPotential_y', 'CentreOfPotential_z', 'Velocity_x',
       'Velocity_y', 'Velocity_z', 'KineticEnergy', 'MechanicalEnergy',
       'TotalEnergy', 'Vmax', 'VmaxRadius', 'Mass', 'MassType_DM',
       'HalfMassProjRad_DM', 'HalfMassRad_DM', 'nodeIndex', 'PosInFile',
       'FileNum', 'Image_ID'],
      dty

In [ ]:
print("Columns with all NaNs:", [col for col in numeric_cols if self.data[col].isna().all()])

In [ ]:
"""
Optimized PyTorch Training Code
- Improved data loading
- Enhanced model architecture
- Optimized training loop
"""

################ MODEL OPTIMIZATION ################

class DMHaloMatterRNN(tcn.Module):
    def __init__(self, 
                input_size=None, 
                hidden_size=None, 
                output_size=None, 
                num_layers=2, 
                dropout_rate=0.2,
                activation='gelu', 
                bidirectional=False, 
                feature_dim=None,
                use_layer_norm=False, 
                return_hidden=False):
        super().__init__()
        self.input_size = input_size if input_size is not None else 64
        self.hidden_size = hidden_size if hidden_size is not None else 128
        self.output_size = output_size if output_size is not None else 1
        self.dropout_rate = dropout_rate
        self.num_layers = num_layers
        self.bidirectional = bidirectional
        self.feature_dim = feature_dim or min(self.hidden_size, self.input_size)
        self.use_layer_norm = use_layer_norm
        self.return_hidden = return_hidden
        
        # Get activation function
        self.activation = self.__get_activation_layer(activation)
        
        # Pre-compute bidirectional factor
        self.output_factor = 2 if self.bidirectional else 1
        
        # Streamlined feature extraction
        self.feature_layer = tcn.Linear(self.input_size, self.feature_dim)
        
        # Optional batch normalization - don't create if not needed
        self.batch_norm = tcn.BatchNorm1d(self.feature_dim)
        
        # Simplified output projection
        intermediate_size = max(4, self.hidden_size // 2)
        self.output_proj1 = tcn.Linear(self.hidden_size * self.output_factor, intermediate_size)
        self.output_proj2 = tcn.Linear(intermediate_size, self.output_size)
        
        # Optional layer normalization
        if self.use_layer_norm:
            self.norm_after_rnn = tcn.LayerNorm(self.hidden_size * self.output_factor)
    
    def __get_activation_layer(self, activation):
        activation_name = activation.lower()
        
        # Simplified activation map with most common activations
        activation_map = {
            'relu': tcn.ReLU(inplace=True),
            'leaky_relu': tcn.LeakyReLU(inplace=True),
            'gelu': tcn.GELU(),
            'swish': tcn.SiLU(inplace=True),
            'silu': tcn.SiLU(inplace=True),
        }
        
        return activation_map.get(activation_name, tcn.GELU())
    
    def forward(self, x):
        """Forward pass (not implemented in base class)."""
        raise NotImplementedError("Forward is Not Implemented")
    
    @tc.jit.ignore
    def process_input(self, x):
        """Optimized input processing"""
        # Handle 2D (single step) vs 3D (sequence) input
        is_sequence = x.dim() == 3
        
        if is_sequence:
            # Batch processing for sequences
            batch_size, seq_len, _ = x.shape
            x = x.reshape(-1, self.input_size)
            x = self.feature_layer(x)
            x = self.activation(x)
            
            # Apply batch norm efficiently
            x = self.batch_norm(x)
            
            # Return to sequence form
            return x.reshape(batch_size, seq_len, self.feature_dim)
        else:
            # Process single step
            x = self.feature_layer(x)
            x = self.activation(x)
            x = self.batch_norm(x)
            return x.unsqueeze(1)  # Add sequence dimension
    
    def apply_weight_init(self):
        """Optimized weight initialization"""
        for m in self.modules():
            if isinstance(m, tcn.Linear):
                tcn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
                if m.bias is not None:
                    tcn.init.zeros_(m.bias)
            elif isinstance(m, tcn.GRU):
                for name, param in m.named_parameters():
                    if 'weight_ih' in name:
                        tcn.init.xavier_uniform_(param)
                    elif 'weight_hh' in name:
                        tcn.init.orthogonal_(param)
                    elif 'bias' in name:
                        tcn.init.zeros_(param)


class GRUDMHaloMapper(DMHaloMatterRNN):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Create GRU layer
        self.gru = tcn.GRU(
            self.feature_dim,
            self.hidden_size,
            self.num_layers,
            batch_first=True,
            dropout=self.dropout_rate if self.num_layers > 1 else 0,
            bidirectional=self.bidirectional
        )
        
        # Apply optimized weight initialization
        self.apply_weight_init()
        
        # Enable cudnn benchmarking for optimal kernels
        if tc.cuda.is_available():
            tc.backends.cudnn.benchmark = True

    def forward(self, x):
        # Process input
        x = self.process_input(x)
        
        # Run through GRU
        features, h_n = self.gru(x)
        
        # Get only the last output efficiently
        features = features[:, -1]
        
        # Apply layer normalization if needed
        if self.use_layer_norm:
            features = self.norm_after_rnn(features)
        
        # Apply functional dropout during training
        if self.training:
            features = F.dropout(features, p=self.dropout_rate, training=True, inplace=True)
        
        # Simplified output projection
        features = self.output_proj1(features)
        features = self.activation(features)
        
        # Final output projection
        output = self.output_proj2(features)
        
        return (output, h_n) if self.return_hidden else output


################ TRAINER OPTIMIZATION ################

class ModelTrainer:
    def __init__(self, model=None, criterion=None, optimizer=None, patience=10, device=None, save_dir=None):
        # Model setup
        self.model = model if model is not None else RNNDMHaloMapper()
        self.device = device if device else ('cuda' if tc.cuda.is_available() else 'cpu')
        self.model = self.model.to(self.device)

        # Criterion and optimizer with better defaults
        self.criterion = criterion or tc.nn.MSELoss()
        self.optimizer = optimizer or tc.optim.AdamW(
            self.model.parameters(), 
            lr=0.001, 
            weight_decay=0.0001,
            eps=1e-8
        )

        # Save and tracking
        self.save_dir = save_dir
        self.metrics_calculator = CustomMetrics()
        self.best_val_loss = float('inf')
        self.patience = patience
        self.patience_counter = 0

        # Training history
        self.history = {
            'train_loss': [],
            'val_loss': [],
            'train_metrics': [],
            'val_metrics': []
        }

        # Configure for performance
        self.__configure_for_performance()

    def __configure_for_performance(self):
        """Configure settings for optimal performance"""
        # Data loaders
        self.train_loader = None
        self.val_loader = None

        # Mixed precision settings
        self.use_amp = self.device == 'cuda'
        if self.use_amp:
            self.scaler = GradScaler()
        
        # Optimize settings for device
        if self.device == 'cuda':
            # Set optimal CUDA settings
            tc.backends.cudnn.benchmark = True
            tc.backends.cudnn.deterministic = False
            # Increase batch size for GPU
            self.log_interval = 25
            self.accumulation_steps = 1
        else:
            # CPU settings
            self.log_interval = 100
            self.accumulation_steps = 1
        
        # Use JIT compilation for CPU if possible
        if self.device == 'cpu' and hasattr(tc, 'compile'):
            try:
                self.model = tc.compile(self.model)
            except:
                pass

    def __optimize_batch_parameters(self):
        """Optimize batch processing parameters"""
        # Auto-tune batching parameters based on the device and model
        if self.device == 'cuda':
            # Find optimal batch size based on model size
            param_count = sum(p.numel() for p in self.model.parameters())
            gpu_mem = tc.cuda.get_device_properties(0).total_memory
            
            if param_count < 1e6:  # Small model
                self.log_interval = 10
            elif param_count < 1e7:  # Medium model
                self.log_interval = 25
            else:  # Large model
                self.log_interval = 50
                
            # Calculate optimal number of workers based on CPU cores
            self.train_loader.num_workers = min(8, os.cpu_count() or 4)
            
            # Set pinned memory for faster data transfer
            if hasattr(self.train_loader, 'pin_memory'):
                self.train_loader.pin_memory = True
                
        else:  # CPU optimizations
            self.log_interval = 100
            if hasattr(self.train_loader, 'num_workers'):
                self.train_loader.num_workers = 0  # Avoid overhead on CPU

    def __train_epoch(self):
        """Optimized training epoch"""
        self.model.train()
        train_loss = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Gradient tracking
        grad_norms = []
        
        # Streamlined progress bar
        pbar = tqdm(self.train_loader, desc="Training")
        
        # Track if metrics need update
        metrics_update_counter = 0
        
        for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
            try:
                # Move data to device efficiently
                batch_features = batch_features.to(self.device, non_blocking=True)
                batch_targets = batch_targets.to(self.device, non_blocking=True)
                
                # Mixed precision training
                if self.use_amp:
                    with autocast(device_type=self.device):
                        outputs = self.model(batch_features)
                        if isinstance(outputs, tuple):
                            outputs = outputs[0]
                        loss = self.criterion(outputs, batch_targets)
                        loss = loss / self.accumulation_steps
                    
                    # Scale gradients
                    self.scaler.scale(loss).backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights with scaler
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        self.optimizer.zero_grad(set_to_none=True)  # More efficient
                else:
                    # Standard CPU training
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    loss = self.criterion(outputs, batch_targets)
                    loss = loss / self.accumulation_steps
                    
                    # Backward pass
                    loss.backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Optional gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights
                        self.optimizer.step()
                        self.optimizer.zero_grad(set_to_none=True)
                
                # Accumulate loss for reporting (unscaled)
                train_loss += loss.item() * self.accumulation_steps
                
                # Update metrics less frequently
                if batch_idx % self.log_interval == 0:
                    metrics_update_counter += 1
                    with tc.no_grad():
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets.detach(), outputs.detach()
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                    
                    # Update progress bar
                    if batch_idx > 0:
                        pbar.set_postfix({'loss': f"{loss.item() * self.accumulation_steps:.5f}"})
            
            except RuntimeError as e:
                logger.error(f"Error in training batch {batch_idx}: {e}")
                continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = train_loss / len(self.train_loader)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics, grad_norms

    def __validate_epoch(self):
        """Optimized validation epoch"""
        self.model.eval()
        val_loss = 0
        val_batches = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Metrics update tracking
        metrics_update_counter = 0
        
        # Progress bar
        pbar = tqdm(self.val_loader, desc="Validation")
        
        with tc.no_grad():
            for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
                try:
                    # Move data to device efficiently
                    batch_features = batch_features.to(self.device, non_blocking=True)
                    batch_targets = batch_targets.to(self.device, non_blocking=True)
                    
                    # Forward pass (simpler without autocast for validation)
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    
                    # Calculate loss
                    loss = self.criterion(outputs, batch_targets)
                    
                    # Track loss
                    val_loss += loss.item()
                    val_batches += 1
                    
                    # Update metrics less frequently
                    if batch_idx % self.log_interval == 0:
                        metrics_update_counter += 1
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets, outputs
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                        
                        # Update progress bar
                        if batch_idx > 0:
                            pbar.set_postfix({'loss': f"{loss.item():.5f}"})
                
                except RuntimeError as e:
                    logger.error(f"Error in validation batch {batch_idx}: {e}")
                    continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = val_loss / max(1, val_batches)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics

    def __create_save_directory(self, model_name):
        """Create directory for saving model artifacts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_dir = os.path.join(self.save_dir, f"{model_name}_{timestamp}")
        os.makedirs(model_dir, exist_ok=True)
        
        best_model_path = os.path.join(model_dir, f"best_{model_name}.pt")
        train_history_path = os.path.join(model_dir, f"{model_name}_history.npy")
        train_history_file = os.path.join(model_dir, f"{model_name}_history.csv")
        
        return {
            'model_dir': model_dir,
            'best_model_path': best_model_path,
            'train_history_path': train_history_path,
            'train_history_file': train_history_file
        }

    def __save_model_checkpoint(self, path):
        """Save model checkpoint efficiently"""
        # Use torch.save with _use_new_zipfile_serialization=True for faster saving
        tc.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, path, _use_new_zipfile_serialization=True)

    def __save_training_progress(self, epoch, train_loss, val_loss, history_file):
        """Save training progress to CSV file"""
        if epoch == 0:
            with open(history_file, 'w') as f:
                f.write('epoch,train_loss,val_loss\n')
        
        with open(history_file, 'a') as f:
            f.write(f'{epoch},{train_loss},{val_loss}\n')

    def __log_metrics(self, train_metrics, val_metrics):
        """Log metrics efficiently"""
        # Only log essential metrics
        essential_metrics = ['mse', 'rmse', 'r2']
        
        train_line = "  ".join([f"{m}: {train_metrics[m]:.5E}" for m in essential_metrics if m in train_metrics])
        val_line = "  ".join([f"{m}: {val_metrics[m]:.5E}" for m in essential_metrics if m in val_metrics])
        
        logger.info(f"Train: {train_line}")
        logger.info(f"Valid: {val_line}")

    def set_loaders(self, X, y):
        """Set up optimized data loaders"""
        # Create dataset and split
        dataset = TensorDataset(tc.FloatTensor(X), tc.FloatTensor(y))
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create optimized data loaders
        num_workers = min(8, os.cpu_count() or 4) if self.device == 'cuda' else 0
        pin_memory = self.device == 'cuda'
        
        self.train_loader = DataLoader(
            train_dataset, 
            batch_size=512, 
            shuffle=True,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )
        
        self.val_loader = DataLoader(
            val_dataset, 
            batch_size=1024,  # Larger batches for validation
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )

    def train(self, num_epochs, model_name, train_loader=None, val_loader=None, X=None, y=None):
        """Optimized training function"""
        # Set up data loaders
        if train_loader and val_loader:
            self.train_loader = train_loader
            self.val_loader = val_loader
        else:
            self.set_loaders(X, y)
        
        # Create save directory
        save_paths = self.__create_save_directory(model_name)
        
        # Initialize learning rate scheduler with better defaults
        scheduler = tc.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3,  # Slightly more patience
            verbose=True
        )
        
        # Training variables
        grad_norms = []
        current_lr = self.optimizer.param_groups[0]['lr']
        
        # Optimize batch parameters
        self.__optimize_batch_parameters()
        
        # Track time and throughput
        start_time = time.time()
        samples_processed = 0
        
        # Training loop
        for epoch in range(num_epochs):
            epoch_start = time.time()
            print(f"\nEpoch [{epoch + 1}/{num_epochs}] | LR: {current_lr:.2e}")
            
            # Training phase
            train_loss, train_metrics, epoch_grad_norms = self.__train_epoch()
            grad_norms.extend(epoch_grad_norms)
            
            # Validation phase
            val_loss, val_metrics = self.__validate_epoch()
            
            # Update learning rate
            scheduler.step(val_loss)
            
            # Check for LR changes
            new_lr = self.optimizer.param_groups[0]['lr']
            if new_lr != current_lr:
                print(f"Learning rate updated: {current_lr:.2e} → {new_lr:.2e}")
                current_lr = new_lr
            
            # Save training progress
            self.__save_training_progress(epoch, train_loss, val_loss, save_paths['train_history_file'])
            
            # Track gradient statistics
            if grad_norms:
                avg_grad_norm = sum(grad_norms[-min(len(grad_norms), len(self.train_loader)):]) / min(len(grad_norms), len(self.train_loader))
                self.history.setdefault('grad_norms', []).append(avg_grad_norm)
            
            # Track samples for throughput calculation
            samples_processed += len(self.train_loader) * self.train_loader.batch_size
            
            # Early stopping check
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                self.__save_model_checkpoint(save_paths['best_model_path'])
                print(f"New best model saved (Val Loss: {val_loss:.5E})")
            else:
                self.patience_counter += 1
            
            # Check for early stopping
            if self.patience_counter >= self.patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_metrics'].append(train_metrics)
            self.history['val_metrics'].append(val_metrics)
            
            # Calculate throughput
            epoch_time = time.time() - epoch_start
            samples_per_second = len(self.train_loader) * self.train_loader.batch_size / epoch_time
            
            # Print summary
            epoch_summary = (
                f"Train Loss: {train_loss:.5E}, Val Loss: {val_loss:.5E}, "
                f"Throughput: {samples_per_second:.1f} it/s, "
                f"Time: {epoch_time:.2f}s"
            )
            
            print(epoch_summary)
            
            # Log metrics occasionally
            if (epoch + 1) % 5 == 0:
                self.__log_metrics(train_metrics, val_metrics)
        
        # Final performance summary
        total_time = time.time() - start_time
        avg_throughput = samples_processed / total_time
        print(f"Training complete. Total time: {total_time:.2f}s, Avg throughput: {avg_throughput:.1f} it/s")
        
        # Save final history
        np.save(save_paths['train_history_path'], self.history)
        
        return self.history, save_paths['model_dir']


################ HYPERPARAMETER OPTIMIZATION ################

def optimize_hyperparameters():
    """Optimized hyperparameters based on target hardware"""
    # Detect device capabilities
    device = 'cuda' if tc.cuda.is_available() else 'cpu'
    
    # Get system info
    cpu_count = os.cpu_count() or 4
    
    # Optimize based on hardware
    if device == 'cuda':
        # GPU configuration
        gpu_mem = tc.cuda.get_device_properties(0).total_memory
        gpu_mem_gb = gpu_mem / (1024**3)
        
        # Scale batch size based on GPU memory
        if gpu_mem_gb > 16:  # High-end GPU
            batch_size = 2048
            hidden_size = 128
        elif gpu_mem_gb > 8:  # Mid-range GPU
            batch_size = 1024
            hidden_size = 64
        else:  # Entry-level GPU
            batch_size = 512
            hidden_size = 32
        
        # Workers based on CPU cores, with limit for GPU data transfer
        num_workers = min(8, cpu_count)
    else:
        # CPU configuration
        batch_size = 256  # Smaller batches for CPU
        hidden_size = 32  # Smaller model for CPU
        num_workers = 0  # Avoid worker overhead on CPU
    
    # Return optimized config
    return {
        'MODEL_TYPE': 'GRU',
        'BATCH_SIZE': batch_size,
        'HIDDEN_SIZE': hidden_size,
        'NUM_WORKERS': num_workers,
        'DEVICE': device,
        'NUM_LAYERS': 1,  # Simpler model is often faster
        'DROPOUT_RATE': 0.1,  # Lower dropout for speed
        'ACTIVATION': 'swish',
        'USE_LAYER_NORM': False,  # Skip layer norm for speed
        'LEARNING_RATE': 0.001,
        'WEIGHT_DECAY': 0.001,
        'OPTIMIZER': 'adamw'  # AdamW often works better than Adam
    }


################ USAGE EXAMPLE ################

def train_optimized_model():
    """Example of using the optimized code"""
    # Get optimized hyperparameters
    config = optimize_hyperparameters()
    
    # Initialize model with optimized configuration
    model = GRUDMHaloMapper(
        input_size=X_train.shape[1],
        hidden_size=config['HIDDEN_SIZE'],
        output_size=1,
        num_layers=config['NUM_LAYERS'],
        dropout_rate=config['DROPOUT_RATE'],
        activation=config['ACTIVATION'],
        bidirectional=False,
        use_layer_norm=config['USE_LAYER_NORM'],
        return_hidden=True
    ).to(config['DEVICE'])
    
    # Create optimizer - AdamW usually better than Adam
    optimizer = tc.optim.AdamW(
        model.parameters(),
        lr=config['LEARNING_RATE'],
        weight_decay=config['WEIGHT_DECAY'],
        eps=1e-8
    )
    
    # Create trainer
    trainer = ModelTrainer(
        model=model,
        criterion=tc.nn.MSELoss(),
        optimizer=optimizer,
        device=config['DEVICE'],
        save_dir=path_manager.model_path
    )
    
    # Create data loaders with optimized batch size
    train_dataset = TensorDataset(tc.FloatTensor(X_train), tc.FloatTensor(y_train))
    val_dataset = TensorDataset(tc.FloatTensor(X_val), tc.FloatTensor(y_val))
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=config['BATCH_SIZE'],
        shuffle=True,
        num_workers=config['NUM_WORKERS'],
        pin_memory=config['DEVICE'] == 'cuda',
        persistent_workers=config['NUM_WORKERS'] > 0,
        prefetch_factor=2 if config['NUM_WORKERS'] > 0 else None
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=config['BATCH_SIZE'] * 2,  # Larger batches for validation
        shuffle=False,
        num_workers=config['NUM_WORKERS'],
        pin_memory=config['DEVICE'] == 'cuda',
        persistent_workers=config['NUM_WORKERS'] > 0,
        prefetch_factor=2 if config['NUM_WORKERS'] > 0 else None
    )
    
    # Train model
    history, model_dir = trainer.train(
        num_epochs=30,
        model_name=config['MODEL_TYPE'],
        train_loader=train_loader,
        val_loader=val_loader
    )
    
    return model, history, model_dir

In [ ]:
        metrics_update_counter = 0
        
        for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
            try:
                # Move data to device efficiently
                batch_features = batch_features.to(self.device, non_blocking=True)
                batch_targets = batch_targets.to(self.device, non_blocking=True)
                
                # Mixed precision training
                if self.use_amp:
                    with autocast(device_type=self.device):
                        outputs = self.model(batch_features)
                        if isinstance(outputs, tuple):
                            outputs = outputs[0]
                        loss = self.criterion(outputs, batch_targets)
                        loss = loss / self.accumulation_steps
                    
                    # Scale gradients
                    self.scaler.scale(loss).backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights with scaler
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        self.optimizer.zero_grad(set_to_none=True)  # More efficient
                else:
                    # Standard CPU training
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    loss = self.criterion(outputs, batch_targets)
                    loss = loss / self.accumulation_steps
                    
                    # Backward pass
                    loss.backward()
                    
                    # Step optimizer at accumulation boundary
                    if (batch_idx + 1) % self.accumulation_steps == 0:
                        # Optional gradient clipping
                        if batch_idx % self.log_interval == 0:
                            grad_norm = tc.nn.utils.clip_grad_norm_(
                                self.model.parameters(), max_norm=1.0
                            )
                            grad_norms.append(grad_norm.item())
                        
                        # Update weights
                        self.optimizer.step()
                        self.optimizer.zero_grad(set_to_none=True)
                
                # Accumulate loss for reporting (unscaled)
                train_loss += loss.item() * self.accumulation_steps
                
                # Update metrics less frequently
                if batch_idx % self.log_interval == 0:
                    metrics_update_counter += 1
                    with tc.no_grad():
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets.detach(), outputs.detach()
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                    
                    # Update progress bar
                    if batch_idx > 0:
                        pbar.set_postfix({'loss': f"{loss.item() * self.accumulation_steps:.5f}"})
            
            except RuntimeError as e:
                logger.error(f"Error in training batch {batch_idx}: {e}")
                continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = train_loss / len(self.train_loader)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics, grad_norms

    def __validate_epoch(self):
        """Optimized validation epoch"""
        self.model.eval()
        val_loss = 0
        val_batches = 0
        
        # Simplified metrics tracking
        epoch_metrics = {
            'mse': 0, 'rmse': 0, 'mae': 0, 'r2': 0
        }
        
        # Metrics update tracking
        metrics_update_counter = 0
        
        # Progress bar
        pbar = tqdm(self.val_loader, desc="Validation")
        
        with tc.no_grad():
            for batch_idx, (batch_features, batch_targets) in enumerate(pbar):
                try:
                    # Move data to device efficiently
                    batch_features = batch_features.to(self.device, non_blocking=True)
                    batch_targets = batch_targets.to(self.device, non_blocking=True)
                    
                    # Forward pass (simpler without autocast for validation)
                    outputs = self.model(batch_features)
                    if isinstance(outputs, tuple):
                        outputs = outputs[0]
                    
                    # Calculate loss
                    loss = self.criterion(outputs, batch_targets)
                    
                    # Track loss
                    val_loss += loss.item()
                    val_batches += 1
                    
                    # Update metrics less frequently
                    if batch_idx % self.log_interval == 0:
                        metrics_update_counter += 1
                        batch_metrics = self.metrics_calculator.calculate_all_metrics(
                            batch_targets, outputs
                        )
                        for metric, value in batch_metrics.items():
                            if metric in epoch_metrics:
                                epoch_metrics[metric] += value
                        
                        # Update progress bar
                        if batch_idx > 0:
                            pbar.set_postfix({'loss': f"{loss.item():.5f}"})
                
                except RuntimeError as e:
                    logger.error(f"Error in validation batch {batch_idx}: {e}")
                    continue
        
        # Calculate average metrics
        num_eval_steps = max(1, metrics_update_counter)
        avg_loss = val_loss / max(1, val_batches)
        avg_metrics = {metric: value / num_eval_steps for metric, value in epoch_metrics.items()}
        
        return avg_loss, avg_metrics

    def __create_save_directory(self, model_name):
        """Create directory for saving model artifacts"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        model_dir = os.path.join(self.save_dir, f"{model_name}_{timestamp}")
        os.makedirs(model_dir, exist_ok=True)
        
        best_model_path = os.path.join(model_dir, f"best_{model_name}.pt")
        train_history_path = os.path.join(model_dir, f"{model_name}_history.npy")
        train_history_file = os.path.join(model_dir, f"{model_name}_history.csv")
        
        return {
            'model_dir': model_dir,
            'best_model_path': best_model_path,
            'train_history_path': train_history_path,
            'train_history_file': train_history_file
        }

    def __save_model_checkpoint(self, path):
        """Save model checkpoint efficiently"""
        # Use torch.save with _use_new_zipfile_serialization=True for faster saving
        tc.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
        }, path, _use_new_zipfile_serialization=True)

    def __save_training_progress(self, epoch, train_loss, val_loss, history_file):
        """Save training progress to CSV file"""
        if epoch == 0:
            with open(history_file, 'w') as f:
                f.write('epoch,train_loss,val_loss\n')
        
        with open(history_file, 'a') as f:
            f.write(f'{epoch},{train_loss},{val_loss}\n')

    def __log_metrics(self, train_metrics, val_metrics):
        """Log metrics efficiently"""
        # Only log essential metrics
        essential_metrics = ['mse', 'rmse', 'r2']
        
        train_line = "  ".join([f"{m}: {train_metrics[m]:.5E}" for m in essential_metrics if m in train_metrics])
        val_line = "  ".join([f"{m}: {val_metrics[m]:.5E}" for m in essential_metrics if m in val_metrics])
        
        logger.info(f"Train: {train_line}")
        logger.info(f"Valid: {val_line}")

    def set_loaders(self, X, y):
        """Set up optimized data loaders"""
        # Create dataset and split
        dataset = TensorDataset(tc.FloatTensor(X), tc.FloatTensor(y))
        train_size = int(0.8 * len(dataset))
        val_size = len(dataset) - train_size
        train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
        
        # Create optimized data loaders
        num_workers = min(8, os.cpu_count() or 4) if self.device == 'cuda' else 0
        pin_memory = self.device == 'cuda'
        
        self.train_loader = DataLoader(
            train_dataset, 
            batch_size=512, 
            shuffle=True,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )
        
        self.val_loader = DataLoader(
            val_dataset, 
            batch_size=1024,  # Larger batches for validation
            shuffle=False,
            num_workers=num_workers,
            pin_memory=pin_memory,
            persistent_workers=num_workers > 0,
            prefetch_factor=2 if num_workers > 0 else None
        )

    def train(self, num_epochs, model_name, train_loader=None, val_loader=None, X=None, y=None):
        """Optimized training function"""
        # Set up data loaders
        if train_loader and val_loader:
            self.train_loader = train_loader
            self.val_loader = val_loader
        else:
            self.set_loaders(X, y)
        
        # Create save directory
        save_paths = self.__create_save_directory(model_name)
        
        # Initialize learning rate scheduler with better defaults
        scheduler = tc.optim.lr_scheduler.ReduceLROnPlateau(
            self.optimizer,
            mode='min',
            factor=0.5,
            patience=3,  # Slightly more patience
            verbose=True
        )
        
        # Training variables
        grad_norms = []
        current_lr = self.optimizer.param_groups[0]['lr']
        
        # Optimize batch parameters
        self.__optimize_batch_parameters()
        
        # Track time and throughput
        start_time = time.time()
        samples_processed = 0
        
        # Training loop
        for epoch in range(num_epochs):
            epoch_start = time.time()
            print(f"\nEpoch [{epoch + 1}/{num_epochs}] | LR: {current_lr:.2e}")
            
            # Training phase
            train_loss, train_metrics, epoch_grad_norms = self.__train_epoch()
            grad_norms.extend(epoch_grad_norms)
            
            # Validation phase
            val_loss, val_metrics = self.__validate_epoch()
            
            # Update learning rate
            scheduler.step(val_loss)
            
            # Check for LR changes
            new_lr = self.optimizer.param_groups[0]['lr']
            if new_lr != current_lr:
                print(f"Learning rate updated: {current_lr:.2e} → {new_lr:.2e}")
                current_lr = new_lr
            
            # Save training progress
            self.__save_training_progress(epoch, train_loss, val_loss, save_paths['train_history_file'])
            
            # Track gradient statistics
            if grad_norms:
                avg_grad_norm = sum(grad_norms[-min(len(grad_norms), len(self.train_loader)):]) / min(len(grad_norms), len(self.train_loader))
                self.history.setdefault('grad_norms', []).append(avg_grad_norm)
            
            # Track samples for throughput calculation
            samples_processed += len(self.train_loader) * self.train_loader.batch_size
            
            # Early stopping check
            if val_loss < self.best_val_loss:
                self.best_val_loss = val_loss
                self.patience_counter = 0
                self.__save_model_checkpoint(save_paths['best_model_path'])
                print(f"New best model saved (Val Loss: {val_loss:.5E})")
            else:
                self.patience_counter += 1
            
            # Check for early stopping
            if self.patience_counter >= self.patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                break
            
            # Update history
            self.history['train_loss'].append(train_loss)
            self.history['val_loss'].append(val_loss)
            self.history['train_metrics'].append(train_metrics)
            self.history['val_metrics'].append(val_metrics)
            
            # Calculate throughput
            epoch_time = time.time() - epoch_start
            samples_per_second = len(self.train_loader) * self.train_loader.batch_size / epoch_time
            
            # Print summary
            epoch_summary = (
                f"Train Loss: {train_loss:.5E}, Val Loss: {val_loss:.5E}, "
                f"Throughput: {samples_per_second:.1f} it/s, "
                f"Time: {epoch_time:.2f}s"
            )
            
            print(epoch_summary)
            
            # Log metrics occasionally
            if (epoch + 1) % 5 == 0:
                self.__log_metrics(train_metrics, val_metrics)
        
        # Final performance summary
        total_time = time.time() - start_time
        avg_throughput = samples_processed / total_time
        print(f"Training complete. Total time: {total_time:.2f}s, Avg throughput: {avg_throughput:.1f} it/s")
        
        # Save final history
        np.save(save_paths['train_history_path'], self.history)
        
        return self.history, save_paths['model_dir']
